In [3]:
!pip install tensorflow
!pip install tflearn
!pip install pipwin
!pipwin install pyaudio
!pip install gtts
!pip install speech_recognition
!pip install playsound

  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=6dc5eb8dadd91e554722e9c5df904cc63ba1df5284102533e67c9910d8ab9ba3
  Stored in directory: c:\users\naeb\appdata\local\pip\cache\wheels\b6\0d\90\0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d
Successfully built termcolor


In [1]:
import random
import json 
import pickle
import numpy as np
import nltk
import tflearn
import tensorflow as tf
import speech_recognition as sr
import playsound
import os

from nltk.stem.lancaster import LancasterStemmer
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
from gtts import gTTS
stemmer = LancasterStemmer()
r = sr.Recognizer()
nltk.download('punkt')

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Naeb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
lemmatizer = WordNetLemmatizer
intents = json.loads(open('data.json').read())
words = []
classes = []
documents = []
ignore_letters = ['?', '!', ',', '.']

for intent in intents['QA']:
  for parttern in intent['patterns']:
    word_list = nltk.word_tokenize(parttern)
    words.extend(word_list)
    documents.append((word_list, intent['type']))
    if intent['type'] not in classes:
      classes.append(intent['type'])


words = [stemmer.stem(w) for w in words if w not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))

training = []
output_empty = [0] * len(classes)

for document in documents:
  bag = []
  word_patterns = document[0]
  word_patterns = [stemmer.stem(w.lower()) for w in word_patterns]
  for word in words:
    bag.append(1) if word in word_patterns else bag.append(0)
  output_row = list(output_empty)
  output_row[classes.index(document[1])] = 1
  training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:, 0])
train_y = list(training[:, 1])
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]), ), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


C:\Users\Naeb\AppData\Local\Temp\ipykernel_6284\2494239605.py:36: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)
C:\Users\Naeb\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\gradient_descent.py:108: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


Train on 1907 samples
Epoch 1/200
1907/1907 [==============================] - 4s 2ms/sample - loss: 7.4952 - acc: 0.0010
Epoch 2/200
1907/1907 [==============================] - 1s 620us/sample - loss: 7.4850 - acc: 0.0021
Epoch 3/200
1907/1907 [==============================] - 1s 593us/sample - loss: 7.4740 - acc: 0.0037
Epoch 4/200
1907/1907 [==============================] - 1s 582us/sample - loss: 7.4536 - acc: 0.0047
Epoch 5/200
1907/1907 [==============================] - 1s 598us/sample - loss: 7.3870 - acc: 0.0047
Epoch 6/200
1907/1907 [==============================] - 1s 591us/sample - loss: 7.2887 - acc: 0.0047
Epoch 7/200
1907/1907 [==============================] - 1s 589us/sample - loss: 7.2357 - acc: 0.0068
Epoch 8/200
1907/1907 [==============================] - 1s 610us/sample - loss: 7.1929 - acc: 0.0089
Epoch 9/200
1907/1907 [==============================] - 1s 592us/sample - loss: 7.1467 - acc: 0.0105
Epoch 10/200
1907/1907 [==============================] - 1s 6

In [9]:
def clean_up_sentence(sentence):
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [stemmer.stem(w) for w in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words = clean_up_sentence(sentence)
  bag = [0] * len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word == w:
        bag[i] = 1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res = model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD = 0.25
  results = [[i, r] for i, r in enumerate(res) if r> ERROR_THRESHOLD]
  
  results.sort(key=lambda x:x[1], reverse=True)
  return_list = []
  for r in results:
    return_list.append({'type': classes[r[0]], 'probability': str(r[1])})
  
  return return_list

def get_response(intents_list, intents_json):
  tag = intents_list[0]['type']
  list_of_intents = intents_json['QA']
  for i in list_of_intents:
    if i['type'] == tag:
      result = random.choice(i['responses'])
      break
  return result

def listen_text():
    with sr.Microphone() as source:
        audio = r.listen(source)
    try:
        text = r.recognize_google(audio,language="en-EN")
    except:
        text = "-"
    return text

def text_to_voice(message): 
    os.remove("output.mp3")
    output = gTTS(message,lang="en", slow=False)
    output.save("output.mp3")
    playsound.playsound('output.mp3', True)

In [ ]:
import time
print('Bot: Hi! talk to me about you want!')
text_to_voice('Hi! talk to me about you want!')
while True:
    message = listen_text()
    if (message == '-'):
        res = "Sorry! I can't hear you"
    else:
        print('You: ', format(message))
        ints = predict_class(message)
        res = get_response(ints, intents)
    text_to_voice(res)
    print('Bot: ', format(res))
    time.sleep(2)

Bot: Hi! talk to me about you want!
You:  hi
Bot:  Hi human
You:  Caillou
Bot:  That is not very nice
You:  heinous
Bot:  How rude
You:  I'm not talking to you
Bot:  No problem
You:  will you understand me
Bot:  I read you loud and clear!
Bot:  Sorry! I can't hear you
Bot:  Sorry! I can't hear you
